Question1

In [41]:
import numpy as np
input = np.array([[2,0,2,1,3,2],[0,2,0,2,2,2],
        [1,0,1,3,1,1],[0,0,1,1,1,0],
        [0,1,3,4,1,0],[0,1,0,0,5,2]])
input

array([[2, 0, 2, 1, 3, 2],
       [0, 2, 0, 2, 2, 2],
       [1, 0, 1, 3, 1, 1],
       [0, 0, 1, 1, 1, 0],
       [0, 1, 3, 4, 1, 0],
       [0, 1, 0, 0, 5, 2]])

In [42]:
npad = ((1,1),(1,1))
padd = np.pad(input,npad,'constant',constant_values=(0))
padd

array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 2, 0, 2, 1, 3, 2, 0],
       [0, 0, 2, 0, 2, 2, 2, 0],
       [0, 1, 0, 1, 3, 1, 1, 0],
       [0, 0, 0, 1, 1, 1, 0, 0],
       [0, 0, 1, 3, 4, 1, 0, 0],
       [0, 0, 1, 0, 0, 5, 2, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]])

In [44]:
filter = np.array([[1,0,0,],[0,1,0,],[0,0,1]])
a = padd.shape[0]-filter.shape[0]+1
b = padd.shape[1]-filter.shape[1]+1
result2 = []
for rn in range(a):
    for cn in range(b):
        result1 = padd[rn:rn+filter.shape[0], cn:cn+filter.shape[1]]*filter
        result2.append(np.sum(result1))

In [45]:
result = np.array(result2).reshape(a,b)
result

array([[ 4,  0,  4,  3,  5,  2],
       [ 0,  5,  3,  5,  4,  5],
       [ 1,  1,  4,  4,  3,  3],
       [ 1,  4,  5,  3,  4,  1],
       [ 1,  1,  3, 10,  4,  1],
       [ 0,  1,  1,  3,  9,  3]])

In [47]:
mx_pool = np.array([[5,5,5],[4,5,4],[1,10,9]])
mx_pool

array([[ 5,  5,  5],
       [ 4,  5,  4],
       [ 1, 10,  9]])

코드참조 : 
https://everyday-deeplearning.tistory.com/entry/%ED%8C%8C%EC%9D%B4%EC%8D%AC%EC%9C%BC%EB%A1%9C-%EB%94%A5%EB%9F%AC%EB%8B%9D%ED%95%98%EA%B8%B0-CNNConvolution-Neural-Network

In [ ]:
Question2

In [48]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init
import torch.nn as nn

In [49]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [50]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [51]:
#MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [52]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [53]:
# CNN Model
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=4, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2))
        #outputsize=13
        
        self.layer2 = nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2))
        
        #outputsize=6
    
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            torch.nn.MaxPool2d(2))

        #outputsize=1
        
        self.fc1 = torch.nn.Linear(1 * 1 * 128, 625)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [54]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [55]:
# train my model
total_batch = len(data_loader)
model.train()    # set the model to train mode (dropout=True)

for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))


[Epoch:    1] cost = 0.304375887
[Epoch:    2] cost = 0.106733769
[Epoch:    3] cost = 0.0809787363
[Epoch:    4] cost = 0.0652073026
[Epoch:    5] cost = 0.0545630939
[Epoch:    6] cost = 0.0457553752
[Epoch:    7] cost = 0.0387457348
[Epoch:    8] cost = 0.0341472141
[Epoch:    9] cost = 0.0300716814
[Epoch:   10] cost = 0.027639294
[Epoch:   11] cost = 0.0226430595
[Epoch:   12] cost = 0.0207825806
[Epoch:   13] cost = 0.017917743
[Epoch:   14] cost = 0.0163265169
[Epoch:   15] cost = 0.0155954603


In [56]:
# Test model and check accuracy
with torch.no_grad():
    model.eval()    # set the model to evaluation mode (dropout=False)

    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

C:\Users\Duck\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\Duck\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9150000214576721
